In [4]:
'''
Backtesting is the process of applying a trading strategy, predictive model, 
or analytical method to historical data to evaluate its accuracy and performance.
'''
# utility library
from datetime import date, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
from sys import modules
from os import listdir
import btalib
import ta

# ploting library
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# machine learning library
from pmdarima.arima import AutoARIMA
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error

# alpaca api
from time import sleep 
from alpaca_trade_api.rest import REST, TimeFrame
from alpaca_trade_api.stream import Stream
from alpaca_trade_api.common import URL
import alpaca_trade_api as tradeapi

# user accounts
from local_settings import alpaca_paper

In [5]:
from enum import Enum
class TimeFrameUnit(Enum):
    Minute = "Min"
    Hour = "Hour"
    Day = "Day"

class TimeFrame:
    def __init__(self, amount: int, unit: TimeFrameUnit):
        self.validate(amount, unit)
        self.__amount = amount
        self.__unit = unit

    @property
    def amount(self):
        return self.__amount

    @amount.setter
    def amount(self, value: int):
        self.validate(value, self.__unit)
        self.__amount = value

    @property
    def unit(self) -> TimeFrameUnit:
        return self.__unit

    @unit.setter
    def unit(self, value: TimeFrameUnit):
        self.validate(self.__amount, value)
        self.__unit = value

    # using "value" field for backwards compatibility
    @property
    def value(self):
        return f"{self.__amount}{self.__unit.value}"

    def __str__(self):
        return self.value

    @staticmethod
    def validate(amount: int, unit: TimeFrameUnit):
        if amount <= 0:
            raise ValueError("Amount must be a positive integer value.")

        if unit == TimeFrameUnit.Minute and amount > 59:
            raise ValueError("Second or Minute units can only be " +
                             "used with amounts between 1-59.")

        if unit == TimeFrameUnit.Hour and amount > 23:
            raise ValueError("Hour units can only be used with amounts 1-23")

In [6]:
# connecting alpaca_trade_api
api_key = alpaca_paper['api_key']
api_secret = alpaca_paper['api_secret']
alpaca_endpoint = alpaca_paper['alpaca_endpoint']

api = tradeapi.REST(api_key,api_secret,alpaca_endpoint,api_version='v2')

ORDERS_URL = "https://paper-api.alpaca.markets/v2/orders"
HEADERS = {"APCA-API-KEY-ID":api_key, "APCA-API-SECRET-KEY": api_secret}


# get account status
account = api.get_account()

# Fetch data -----------------------------------------------------------------------------------------------
tickers = ["AAPL", "SPY"]

# Get time
starting_date = "2015-06-07"
ending_date = "2019-06-07"

freq = TimeFrame(1, TimeFrameUnit.Day)

bars = {}
quotes = {}
trades = {}
for ticker in tickers:
    bar = api.get_bars(ticker, freq, starting_date, ending_date, adjustment='raw').df
    quote = api.get_quotes(ticker, starting_date, ending_date, limit=10).df
    trade = api.get_trades(ticker, starting_date, ending_date, limit=10).df
    bars[ticker] = bar
    quotes[ticker] = quote
    trades[ticker] = trade

In [7]:
# Select one stocks to analyze
stock = "SPY" # modify to select stock
df = bars[stock] 
df = df.reset_index()
df['timestamp'] = df['timestamp'].dt.date
df.set_index('timestamp')

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01,209.42,210.8200,209.1100,210.680,97858418,337780,209.921410
2015-12-02,210.63,211.0000,208.2300,208.529,108069059,367013,209.563055
2015-12-03,208.90,209.1500,204.7511,205.610,166224154,546768,206.878936
2015-12-04,206.10,209.9700,205.9300,209.620,192878747,556731,208.178631
2015-12-07,209.22,209.7295,207.2000,208.350,102027111,374705,208.276128
...,...,...,...,...,...,...,...
2019-06-03,275.30,276.5521,273.0900,274.610,96584146,492872,274.759710
2019-06-04,277.11,280.6800,276.6200,280.490,77767889,377640,278.845031
2019-06-05,282.34,282.9900,280.3200,282.870,71374498,327592,281.929760


In [8]:
# Calcultae indicators
df['rsi'] = ta.momentum.RSIIndicator(df['close'], window = 14).rsi()
df['sma']= btalib.sma(df['close'], period=10).df

In [9]:
# Create figure with secondary y-axis
def candlestick(df):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=False, vertical_spacing=0.1)

    # include candlestick with rangeselector
    fig.append_trace(go.Candlestick(x=df['timestamp'], open=df['open'], high=df['high'], 
    low=df['low'], close=df['close']), row=1, col=1)

    # include a go.Bar trace for volumes
    # fig.add_trace(go.Bar(x=df.index, y=df['volume']), secondary_y=False)

    fig.append_trace(go.Scatter(x=df.index,y=df['rsi'],
        name='RSI',
        mode='lines'), 
        row=2, col=1)

    fig.update_xaxes(row=1, col=1, rangeslider_thickness=0.05)
    fig.update_layout(title= stock,
                      width=900, height=900,
                      xaxis_title='Date',
                      yaxis_title='Prices')
    fig.show()
candlestick(df)

# Simulation

In [10]:
# create 20 days simple moving average column
df['3_SMA'] = df['close'].rolling(window = 3, min_periods = 1).mean()
df['3_SMA_l'] = df['close'].rolling(window = 3, min_periods = 1).min()
df['3_SMA_h'] = df['close'].rolling(window = 3, min_periods = 1).max()

In [11]:
fig = make_subplots()
# include candlestick with rangeselector
fig.add_trace(go.Candlestick(x=df['timestamp'], open=df['open'], high=df['high'], low=df['low'], close=df['close']))
fig.add_trace(go.Scatter(x=df['timestamp'],y=df['close'], name = 'price', mode = 'lines'))
fig.add_trace(go.Scatter(x=df['timestamp'],y=df['3_SMA_l'], name = 'SMA - low', mode = 'lines'))
fig.add_trace(go.Scatter(x=df['timestamp'],y=df['3_SMA_h'], name = 'SMA - high', mode = 'lines'))
fig.add_trace(go.Scatter(x=df['timestamp'],y=df['3_SMA'], name = 'SMA', mode = 'lines'))
fig.update_layout(title= stock,
                      width=900, height=900,
                      xaxis_title='Date',
                      yaxis_title='Prices')
fig.show()

In [17]:
amount = 200
cash = 100000
shares = 0
log = []

for i in range(len(df)):
        if cash > 0 and shares < 500 and df["low"][i] <= df['3_SMA_l'][i] <= df['high'][i]:
                # buy
                shares += amount
                cash -= amount * df['3_SMA_l'][i]    
        elif shares and df["low"][i] <= df['3_SMA_h'][i] <= df['high'][i]:
                # sell
                cash += shares * df['3_SMA_h'][i]
                shares = 0
        equity = cash + shares * df['close'][i]
        log.append(equity)

fig = make_subplots(rows=2, cols=1, shared_xaxes=False, vertical_spacing=0.1)

# include candlestick with rangeselector
fig.append_trace(go.Candlestick(x=df['timestamp'], open=df['open'], high=df['high'], 
low=df['low'], close=df['close']), row=1, col=1)

fig.append_trace(go.Scatter(x=df['timestamp'],y=log), 
        row=2, col=1)

fig.update_xaxes(row=1, col=1, rangeslider_thickness=0.05)
fig.update_layout(title= stock,
                      width=900, height=900,
                      xaxis_title='Date',
                      yaxis_title='Prices')
fig.show()